In [1]:
from google.colab import drive
drive.mount('/content/drive')

path_to_drive = "/content/drive/MyDrive/Colab Notebooks"

Mounted at /content/drive


# spec

In [4]:
!nvidia-smi

Sun Sep  5 14:23:57 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 63
model name	: Intel(R) Xeon(R) CPU @ 2.30GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2299.998
cache size	: 46080 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 avx2 smep bmi2 erms invpcid xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs
bogomips	: 4599.99
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 bits virtual
power management:

processor	:

# Get GitHub repository files and install python packages

In [6]:
!git clone https://github.com/Team-M1/badwords-classifier-train -b WJ
%cd badwords-classifier-train
!pip install -r requirements.txt

Cloning into 'badwords-classifier-train'...
remote: Enumerating objects: 221, done.
remote: Counting objects: 100% (221/221), done.
remote: Compressing objects: 100% (191/191), done.
remote: Total 221 (delta 122), reused 126 (delta 28), pack-reused 0
Receiving objects: 100% (221/221), 2.80 MiB | 14.57 MiB/s, done.
Resolving deltas: 100% (122/122), done.
/content/badwords-classifier-train
     |████████████████████████████████| 264 kB 5.4 MB/s 
     |████████████████████████████████| 282 kB 42.4 MB/s 
     |████████████████████████████████| 2.8 MB 36.2 MB/s 
     |████████████████████████████████| 919 kB 33.3 MB/s 
     |████████████████████████████████| 243 kB 50.4 MB/s 
     |████████████████████████████████| 119 kB 52.3 MB/s 
     |████████████████████████████████| 50 kB 6.4 MB/s 
     |████████████████████████████████| 895 kB 38.0 MB/s 
     |████████████████████████████████| 636 kB 38.2 MB/s 
     |████████████████████████████████| 3.3 MB 39.7 MB/s 
     |██████████████████████████

# model and tokenizer

In [15]:
import numpy as np
import pandas as pd
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer


model = AutoModelForSequenceClassification.from_pretrained(f"{path_to_drive}/koelectra-small-v3-discriminator-1/checkpoint-26000")
tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-small-v3-discriminator")

# datasets

In [8]:
test_df = pd.read_csv("data/test.csv")
test_data = test_df["content"]
test_label = test_df["labels"]

speed_test_data = test_data.head(50).tolist()

# speed

In [16]:
model.to("cpu")
model.eval()
print()

In [17]:
%%timeit

with torch.no_grad():
    tokens = tokenizer(speed_test_data, padding="max_length", truncation=True, return_tensors="pt")
    _ = model(**tokens)


1 loop, best of 5: 15.7 s per loop


# f1

In [18]:
from data_loader import get_data_loaders


_, _, test_loader = get_data_loaders(tokenizer, use_imbalanced=False, device="cuda")

Using custom data configuration default-8583e4e44ee1f263
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-8583e4e44ee1f263/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff)
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-8583e4e44ee1f263/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff/cache-ae32d7e6089d916b.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-8583e4e44ee1f263/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff/cache-41fff839581df280.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-8583e4e44ee1f263/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff/cache-f6046a55d6638597.arrow


In [19]:
from torchmetrics import F1
from tqdm.auto import tqdm


f1_score = F1(num_classes=3, average="macro").cuda()
model.cuda()
model.eval()

with torch.no_grad():
    for batch in tqdm(test_loader):
        labels = batch.pop("labels")
        output = model(**batch)
        logits = output.logits
        pred = torch.argmax(logits, dim=1)
        f1_score(pred, labels)

score = f1_score.compute()
print(score.item())

  0%|          | 0/157 [00:00<?, ?it/s]

0.9179062247276306
